# Homework Starter — Stage 15: Orchestration & System Design
Complete the sections below. Keep your answers concise and focused on orchestration readiness.

## 1) Project Task Decomposition
List 4–8 tasks. Add more rows as needed.

In [ ]:

import pandas as pd
import argparse, logging, sys, pickle
from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

tasks = pd.DataFrame({
    'task': [
        'ingest',
        'clean',
        'feature_engineer',
        'train_model',
        'evaluate',
        'report'
    ],
    'inputs': [
        '/data/raw/*.csv',
        'raw_ingested.csv',
        'clean.csv',
        'features.csv',
        'features.csv + model.pkl',
        'metrics.json + model.pkl'
    ],
    'outputs': [
        'raw_ingested.csv',
        'clean.csv',
        'features.csv',
        'model.pkl',
        'metrics.json',
        'report.md'
    ],
    'idempotent': [True, True, True, True, True, True]
})

print("=== Task Decomposition ===")
print(tasks)



def train_model(input_path: str, output_path: str, log_path: str) -> None:
    logging.info("[train_model] start")

    df = pd.read_csv(input_path)

    # Fill missing values with median
    df = df.fillna(df.median(numeric_only=True))

    X = df[['income', 'transactions']].values
    y = df['spend'].values

    model = LinearRegression()
    model.fit(X, y)

    preds = model.predict(X)
    mae = mean_absolute_error(y, preds)
    logging.info("[train_model] MAE = %.4f", mae)

    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    pickle.dump(model, open(output_path, "wb"))
    logging.info("[train_model] model saved at %s", output_path)

    Path(log_path).parent.mkdir(parents=True, exist_ok=True)
    with open(log_path, "w") as f:
        f.write(f"MAE: {mae:.4f}\n")


def main(argv=None):
    parser = argparse.ArgumentParser(description="Train model task")
    parser.add_argument("--input", required=True, help="Processed feature CSV")
    parser.add_argument("--output", required=True, help="Path to save model.pkl")
    parser.add_argument("--log", required=True, help="Path to save metrics log")
    args = parser.parse_args(argv)

    logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)])
    train_model(args.input, args.output, args.log)


if __name__ == "__main__":
    # Example CLI call (for testing)
    main([
        "--input", "../data/processed/features.csv",
        "--output", "../models/model.pkl",
        "--log", "../reports/metrics.txt"
    ])


In [ ]:
from pathlib import Path
import pandas as pd
tasks = pd.DataFrame({
    'task': ['ingest', 'clean', 'train_or_score', 'report'],
    'inputs': ['/data/raw.ext', 'prices_raw.json', 'prices_clean.json', 'model.json'],
    'outputs': ['prices_raw.json', 'prices_clean.json', 'model.json', 'report.txt'],
    'idempotent': [True, True, True, True]
})
tasks

## 2) Dependencies (DAG)
Describe dependencies and paste a small diagram if you have one.

In [ ]:
dag = {
    'ingest': [],
    'clean': ['ingest'],
    'feature_engineer': ['clean'],
    'train_model': ['feature_engineer'],
    'evaluate': ['feature_engineer', 'train_model'],
    'report': ['evaluate']
}

print("\n=== DAG Dependencies ===")
print(dag)


## 3) Logging & Checkpoints Plan
Specify what you will log and where you will checkpoint for each task.

In [ ]:
logging_plan = pd.DataFrame({
    'task': ['ingest', 'clean', 'feature_engineer', 'train_model', 'evaluate', 'report'],
    'log_messages': [
        'start/end, row counts, source URI',
        'start/end, null rates before/after',
        'new feature stats, drift checks',
        'hyperparams, training MAE',
        'eval metrics (MAE, calibration)',
        'report generated path'
    ],
    'checkpoint_artifact': [
        'raw_ingested.csv',
        'clean.csv',
        'features.csv',
        'model.pkl',
        'metrics.json',
        'report.md'
    ]
})

print("\n=== Logging & Checkpoints ===")
print(logging_plan)


## 4) Right-Sizing Automation
Which parts will you automate now? Which stay manual? Why?

*(Write your rationale here.)*


### Automate Now
- **Ingest**: Reading raw CSVs is simple, deterministic, and should be scripted for reproducibility.  
- **Clean**: Cleaning rules (null handling, type casting) are stable and benefit from automated checkpoints.  
- **Feature Engineering**: Core features (`spend_income_ratio`, `rolling_spend_mean`) are consistent transformations, so automating ensures no manual mistakes.  
- **Train Model**: Training with fixed parameters is deterministic and should always be automated to ensure reproducible model artifacts.  
- **Evaluate**: Computing metrics (MAE, accuracy) is repeatable and should be logged automatically.

### Keep Manual (for now)
- **Report Writing**: While generating a basic metrics report can be automated, higher-level narrative and visual interpretation are better handled manually at this stage.  
- **DAG Visualization**: The DAG structure may evolve, so maintaining a static diagram manually is sufficient for now.

### Rationale
- **Automated** steps are those that need to run consistently whenever new data arrives or experiments are rerun.  
- **Manual** steps involve interpretation, storytelling, or visual presentation, which still benefit from human judgment and may change frequently.  
- This balance allows us to focus engineering effort where reproducibility matters most (data → model → metrics), while keeping reporting flexible.


## 5) (Stretch) Refactor One Task into a Function + CLI
Use the templates below.

In [ ]:




def train_model(input_path: str, output_path: str, log_path: str) -> None:
    logging.info("[train_model] start")

    df = pd.read_csv(input_path)

    # Fill missing values with median
    df = df.fillna(df.median(numeric_only=True))

    X = df[['income', 'transactions']].values
    y = df['spend'].values

    model = LinearRegression()
    model.fit(X, y)

    preds = model.predict(X)
    mae = mean_absolute_error(y, preds)
    logging.info("[train_model] MAE = %.4f", mae)

    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    pickle.dump(model, open(output_path, "wb"))
    logging.info("[train_model] model saved at %s", output_path)

    Path(log_path).parent.mkdir(parents=True, exist_ok=True)
    with open(log_path, "w") as f:
        f.write(f"MAE: {mae:.4f}\n")


def main(argv=None):
    parser = argparse.ArgumentParser(description="Train model task")
    parser.add_argument("--input", required=True, help="Processed feature CSV")
    parser.add_argument("--output", required=True, help="Path to save model.pkl")
    parser.add_argument("--log", required=True, help="Path to save metrics log")
    args = parser.parse_args(argv)

    logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)])
    train_model(args.input, args.output, args.log)


if __name__ == "__main__":
    # Example CLI call (for testing)
    main([
        "--input", "../data/processed/features.csv",
        "--output", "../models/model.pkl",
        "--log", "../reports/metrics.txt"
    ])


### Optional: Simple Retry Wrapper (fill in)
Add a small retry with linear backoff to harden a task.

In [ ]:
import time
def retry(n_tries=3, delay=0.2):
    def wrapper(fn, *args, **kwargs):
        # TODO: implement try/except loop with sleep backoff
        return fn(*args, **kwargs)
    return wrapper